In [1]:
!pip install -q geemap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.1 MB/s eta 0:00:00


In [2]:
import ee
import geemap

In [3]:
import ee
ee.Authenticate()

In [4]:
ee.Initialize(project='lips-482716')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


### Visualizing the 'Cachoeira Seca' Indigenous Land Area

To visualize the 'Cachoeira Seca' indigenous land, we followed these steps:

1.  **Load the GeoJSON File**: We first loaded the `cachoeira_seca_georeferenced.geojson` file, which contains the geographical data for the indigenous land.


2.  **Convert to Earth Engine Object**: The GeoJSON data was then converted into an Earth Engine feature collection using `geemap.geojson_to_ee()`. This allows us to work with the data within the Earth Engine environment.


In [5]:
output_geojson_path = '/content/cachoeira_seca_georeferenced.geojson'
cachoeira_seca_feature = geemap.geojson_to_ee(output_geojson_path)

3.  **Define Region of Interest (ROI)**: We extracted the bounding box (`roi`) of the `cachoeira_seca_feature` to define the spatial extent of our area of interest. This ROI is useful for centering maps and defining the region for static image generation.


In [7]:

roi = cachoeira_seca_feature.geometry().bounds()

5.  **Interactive Map Visualization with `geemap`**: To provide an interactive view of the data, we used `geemap.Map()`.
    *   An interactive map instance was created.
    *   Visualization parameters (`vis_params_feature`, `vis_params_roi`) were defined to style the indigenous land area and its bounding box (ROI), using a purple color.
    *   Both the `cachoeira_seca_feature` and the `roi` were added as layers to the map.
    *   The map was then centered on the ROI with a zoom level of 8, allowing for a clear interactive exploration of the area.

In [17]:
map = geemap.Map()

# Define a visualization for the feature layer
vis_params_feature = {
        'color': '800080',  # Purple color
        'pointSize': 5,
        'pointShape': 'circle',
        'width': 2           # Line width
    }
vis_params_roi = {
        'color': '800080',  # Purple color
        'pointSize': 5,
        'pointShape': 'rectangle',
        'width': 2           # Line width
    }
# Add layers to the interactive map for display
map.addLayer(cachoeira_seca_feature, vis_params_feature, 'cachoeira_seca KML')
map.addLayer(roi, vis_params_roi, 'cachoeira_seca roi')

map.centerObject(roi, 8)

#map

👉 [Open interactive map](https://github.com/rociolopeth/Amazonia-deforestation/blob/main/docs/cachoeira_seca_map.html)


In [15]:
def extract_map_biomas (year,asset_path ):
  # load ImageCollection and filter
  mapbiomas_ic = (
      ee.ImageCollection(asset_path)
      .filter(ee.Filter.eq('collection_id', 10))
      .filter(ee.Filter.eq('version', 'v1'))
      .filter(ee.Filter.eq('year', year))
  )

  # extract biomas
  mapbiomas = mapbiomas_ic.first()
  forest = (
      mapbiomas.select('classification').eq(3)
      .Or(mapbiomas.select('classification').eq(5))
      .Or(mapbiomas.select('classification').eq(6))
  )
  agriculture = (
      mapbiomas.select('classification').eq(18)
      .Or(mapbiomas.select('classification').eq(39))
      .Or(mapbiomas.select('classification').eq(40))
      .Or(mapbiomas.select('classification').eq(41))

  )
  pasture = mapbiomas.select('classification').eq(15)

  return forest, agriculture, pasture